# jieba pathon API

In [ ]:
import jieba

In [ ]:
# 搜索引擎模式
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  

In [ ]:
dict(seg_list)

In [ ]:
chilist = list(seg_list)

In [ ]:
chilist

# 分类器基类

In [ ]:
import jieba
from pysqlite2 import dbapi2 as sqlite
import re
import math
import os
import string

In [ ]:
# 提取特征函数,返回特征字典
def separatewords(text):
    # 搜索引擎模式
    seg_list = jieba.cut_for_search(text)  
    words_list = list(seg_list)
    words = [s for s in words_list if len(s)>2 ]
    return dict([(w,1) for w in words])

In [ ]:
# 分类器基类
class classifier:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        # 特征/分类组合： python:{bad:1,good:3}
        self.fc = {}
        # Counts of documents in each category
        # 分类文档数量
        self.cc = {}
        # 提取特征函数
        self.getfeatures = getfeatures
        #
        self.dbfile = filename

    def setdb(self,dbfile):
        self.dbfile = dbfile
        #
        self.con = sqlite.connect(dbfile)    
        self.con.execute('create table if not exists fc(feature,category,count)')
        self.con.execute('create table if not exists cc(category,count)')
        self.con.execute('create index if not exists featureidx on fc(feature)')
        self.con.execute('create index if not exists fccategoryidx on fc(category)')
        self.con.execute('create index if not exists cccategoryidx on cc(category)')

    # 增加 特征/分类组合 数量
    def incf(self,f,cat):
        if self.dbfile == None:
            self.fc.setdefault(f,{})
            self.fc[f].setdefault(cat,0)
            self.fc[f][cat] += 1
        else:
            count = self.fcount(f,cat)
            if count == 0:
                self.con.execute("insert into fc values ('%s','%s',1)" % (f,cat))
            else:
                self.con.execute("update fc set count=%d where feature='%s' and category='%s'" % (count+1,f,cat)) 

    # 增加 分类文档 数量
    def incc(self,cat):
        if self.dbfile == None:
            self.cc.setdefault(cat,0)
            self.cc[cat] += 1
        else:
            count = self.catcount(cat)
            if count==0:
                self.con.execute("insert into cc values ('%s',1)" % (cat))
            else:
                self.con.execute("update cc set count=%d where category='%s'" % (count+1,cat))  

    # 取得 特征/分类组合 数量            
    def fcount(self,f,cat):
        if self.dbfile == None:
            if f in self.fc and cat in self.fc[f]:
                return float(self.fc[f][cat])
            return 0.0
        else:
            res = self.con.execute('select count from fc where feature="%s" and category="%s"'%(f,cat)).fetchone()
            if res == None: return 0
            else: return float(res[0])

    # 取得 分类文档 数量    
    def catcount(self,cat):
        if self.dbfile == None:
            if cat in self.cc:
                return float(self.cc[cat])
            return 0
        else:
            res = self.con.execute('select count from cc where category="%s"' %(cat)).fetchone()
            if res == None: return 0
            else: return float(res[0])

    # 取得 所有文档 数量    
    def totalcount(self):
        if self.dbfile == None:
            return sum(self.cc.values())
        else:
            res = self.con.execute('select sum(count) from cc').fetchone();
            if res == None: return 0
            return res[0]

    # 取得 所有分类的列表
    def categories(self):
        if self.dbfile == None:
            return self.cc.keys()
        else:
            cur = self.con.execute('select category from cc');
            return [d[0] for d in cur]

    # 训练函数
    def train(self,item,cat):
        features = self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)

        # Increment the count for this category
        self.incc(cat)

        #
        if self.dbfile != None:
            self.con.commit()

    # 特征分类概率 = 特征在分类下数量 / 分类文档总数量
    # 因为分类函数返回的是字典，所以 (特征在分类下数量) <= (分类文档总数量)
    # (python | good) <= (good) == ( python|good + snake|good + programmer|good + ...)
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0

        # The total number of times this feature appeared in this 
        # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)   

    # 加权概率 【特征,分类,特征概率函数,假设概率权重,假设概率值】
    # 防止没有遇见的特征分类值为零,预加了 (假设概率权重 * 假设概率值)
    # 让没有见过的特征分类值保持近似中性，有一个计算后的加权概率值
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob = prf(f,cat)
        #print 'basicprob:',basicprob
        # Count the number of times this feature has appeared in
        # all categories
        # 特征在所有分类下的总量
        # 即包含特征的文章总数
        totals = sum([self.fcount(f,c) for c in self.categories()])
        #print 'totals:',totals

        # Calculate the weighted average
        # ((假设权重 * 假设概率) + (特征分类概率 * 特征文章总数) / ( 假设权重 + 特征文章总数)
        #  因 假设概率 <=1 , 特征分类概率 <=1 , 所以 该公式结果 <=1
        bp = ((weight * ap) + (totals * basicprob))/(weight + totals)
        return bp

In [ ]:
def read_and_parse_file(file_path):
    # 去掉标点符号
    delset = string.punctuation
    try:
        if os.path.exists(file_path):
            f = open(file_path, "r")
            lines = f.readlines()
            f.close()
            if len(lines) < 5:
                return None,None,0,None,None
            title = lines[0].strip()
            fields = lines[2].split(':')
            zhan_num = int(fields[2].strip())
            author = fields[1].strip().split(' ')[0].strip()
            content = ''.join(lines[5:-2])
            url = lines[-1].split(' ')[1].strip()
            #
            title = title.translate(None,delset)
            author = author.translate(None,delset)
            content = content.translate(None,delset)
            return title,author,zhan_num,url,content
    except IOError:
        error_file_path = file_path
        print "error_file_path:" , error_file_path
    
    # 默认
    return None,None,0,None,None

In [ ]:
# 监督学习训练函数
def sampletrain(cl,rootdir):
    i = 0
    for parent,dirnames,filenames in os.walk(rootdir):  
        #输出文件信息
        for filename in filenames:                        
            #print "parent is:" + parent
            #print type(parent)
            #print "filename is: " + str(filename).decode('gbk', 'ignore')
            #输出文件路径信息
            #print "the full name of the file is:" + os.path.join(parent,filename) 
            file_class = parent.split("\\")[1]
            #print file_class
            full_name = os.path.join(parent,filename) 
            i += 1
            print str(i) + " ,full_name is " + str(full_name).decode('gbk', 'ignore')
            # 读取文件
            title,author,zhan_num,url,content = read_and_parse_file(full_name)
            if title == None:
                continue
            #
            cl.train("%s %s %s" % (str(title),str(author),str(content)),file_class.decode('gbk', 'ignore')) 

In [ ]:
rootdir = 'E:/project/pychram/zhihu/zhihu-python/text'

In [ ]:
cl = classifier(separatewords)

In [ ]:
sampletrain(cl,rootdir)

In [ ]:
cl.train('the quick brown fox jump over the lazy dog','good')
cl.train('make quick money in the on1ine casio ','bad')
cl.fcount('quick' , 'good')
cl.catcount('good')
cl.catcount('bad')
cl.totalcount()
cl.fcount('the' , 'good')
cl.fc
sampletrain(cl)
cl.fprob('quick','good')
cl.weightedprob('money','good',cl.fprob)
cl.fprob('money','good')
cl.weightedprob('money','good',cl.fprob)
cl.fprob('money','bad')
cl.weightedprob('money','bad',cl.fprob)

In [ ]:
cl.totalcount()

In [ ]:
cl.cc

In [ ]:
cl.fc['台北市'.decode('UTF-8','ignore')]

In [ ]:
cl.fprob('台北市'.decode('UTF-8','ignore'),'百科'.decode('UTF-8','ignore'))

In [ ]:
cl.weightedprob('台北市'.decode('UTF-8','ignore'),'百科'.decode('UTF-8','ignore'),cl.fprob)

In [ ]:
cl.weightedprob('台北市'.decode('UTF-8','ignore'),'IT'.decode('UTF-8','ignore'),cl.fprob)

In [ ]:
cl.weightedprob('台北市'.decode('UTF-8','ignore'),'Python'.decode('UTF-8','ignore'),cl.fprob)

In [ ]:
cl.fprob('台北市'.decode('UTF-8','ignore'),'Python'.decode('UTF-8','ignore'))

In [ ]:
cl.fprob('Ironymode','IT')

In [ ]:
cl.fc['台北市'.decode('UTF-8','ignore')]

In [ ]:
class naivebayes(classifier):
    # P(分类|文档) = P(文档|分类) * P(分类) / P(文档)
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds = {}

    # 各个特征分类概率连乘作为文档分类概率 
    # (f1|cate * f2|cate * f2|cate * ...) == P(文档|分类)
    def docprob(self,item,cat):
        features = self.getfeatures(item)   

        # Multiply the probabilities of all the features together
        p = 1
        for f in features: p *= self.weightedprob(f,cat,self.fprob)
        return p     

    # P(分类|文档) = (分类文档数量/文档总数) * 文档分类概率 / 文档总数
    def prob(self,item,cat):
        # 分类概率 = (分类文档数量/文档总数) == P(分类)  
        catprob = self.catcount(cat)/self.totalcount()
        # 文档分类概率 = P(文档|分类)
        docprob = self.docprob(item,cat)
        return docprob * catprob    

    # 设置分类的阀值
    def setthreshold(self,cat,t):
        self.thresholds[cat]=t

    # 取得分类的阀值,默认 1:1
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]  

    # 查找分类
    def classify(self,item,default=None):
        probs = {}
        # Find the category with the highest probability
        max = 0.0
        for cat in self.categories():
            probs[cat] = self.prob(item,cat)
            if probs[cat] > max: 
                max = probs[cat]
                best = cat

        # best 目前是最大可能性的分类
        # Make sure the probability exceeds threshold*next best
        for cat in probs:
            if cat == best: continue
            # 最佳分类 < (其他任何分类 * 最佳分类阀值) 
            # 不能当作最佳分类，返回 默认分类             
            if probs[cat] * self.getthreshold(best) > probs[best]: return default

        # 最佳分类 >= (其他任何分类 * 最佳分类阀值) 
        # 返回最佳分类
        return best

In [ ]:
cl = naivebayes(separatewords)
sampletrain(cl,rootdir)

In [ ]:
cl.prob('台北市'.decode('UTF-8','ignore'),'百科'.decode('UTF-8','ignore'))

In [ ]:
cl.prob('台北市'.decode('UTF-8','ignore'),'IT'.decode('UTF-8','ignore'))

In [ ]:
judge = cl.classify('怎样区分罗氏虾、基围虾、大头虾、青虾、九节虾、花虾、竹节虾、对虾、沙虾、虎虾……各种虾 又称烟熏食品香精 烟熏调味品可分为原生型和派生型两大类 但传统烟熏法温度很难控制 怎样区分大闸蟹、毛蟹、青蟹、梭子蟹、河蟹、红鲟、螃蜞，帝王蟹，松叶蟹等等螃蟹'.decode('UTF-8','ignore'),default='unknown')

In [ ]:
print judge

In [ ]:
judge = cl.classify('''
支付意愿法。简单说，就是研究者进行问卷调查统计，如果能够减少你百分之一的死亡率，你愿意付多少钱？这个方法算出来的数字普遍比第一种要高，不过人们在回答问卷时对假想风险的评定往往与面临真实死亡风险时的反映有所不同，所以可能不如第一种客观。

第三种方法是人力资本价值法。也就是开头提到的按工资来计算人命价值。这个方法的最大缺陷是低估了人的生命价值，因为生命的意义不仅仅是工作和赚工资。比如一个乞丐一生没有收入，他的生命是不是就没有价值呢？显然不是。

为生命定价的主要目的是用于政策研究。比如，美国官方各个部门对生命的定价都不一样，美国环保部的定价是910万，食品药品监督局是790万，交通部是940万。以这个定价为标准，很多政策就很容易决定。比如修缮高速公路可以减少10个交通事故死亡人数，需要花1亿美元，交通部一算，十个人值9400万美元，就不会去修这条路。

问题中的法国巴黎人和叙利亚人，生命价格必然是不一样的。考虑到美国和中国的人命价格差距已经相差约50倍，那么法国首都和叙利亚前线的人命价格恐怕相差会更多。

你看，人的生命在经济学家眼里比法律工作者那里更贵。而比起政客来，大家都显得情深意重。举个例子，大家都知道烟草对身体有害，吸烟致男性平均减寿7.13年，女性平均减寿4.5年。有人说，不禁烟是因为国家需要收税，可是事实上，政府在烟草上的税收，是少于因为吸烟造成的医疗开销和生产率损失的。那么，既然对健康不利同时又消耗医疗资源，为什么政府不干脆禁烟呢？


''',default='unknown')

In [ ]:
print judge

In [ ]:
cl.prob('台北市'.decode('UTF-8','ignore'),'人生'.decode('UTF-8','ignore'))

In [ ]:
cl.prob('台北市'.decode('UTF-8','ignore'),'百科'.decode('UTF-8','ignore'))

In [ ]:
cl.prob('熏肉'.decode('UTF-8','ignore'),'人生'.decode('UTF-8','ignore'))

In [ ]:
cl.prob('又称烟熏食品香精'.decode('UTF-8','ignore'),'食品'.decode('UTF-8','ignore'))

In [ ]:
class fisherclassifier(classifier):

    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.minimums = {}

    # 特征概率
    def cprob(self,f,cat):
        # The frequency of this feature in this category    
        # 特征分类概率 = 特征在分类下数量 / 分类文档总数量
        clf = self.fprob(f,cat)
        if clf==0: return 0

        # The frequency of this feature in all the categories
        # 特征在所有分类下的总概率
        freqsum = sum([self.fprob(f,c) for c in self.categories()])

        # The probability is the frequency in this category divided by
        # the overall frequency
        # 特征分类概率 / 特征在所有分类下的总概率
        p = clf/(freqsum)

        return p


    def fisherprob(self,item,cat):
        # Multiply all the probabilities together
        p = 1
        features = self.getfeatures(item)
        for f in features:
            p *= (self.weightedprob(f,cat,self.cprob))

        # Take the natural log and multiply by -2
        fscore=-2*math.log(p)

        # Use the inverse chi2 function to get a probability
        return self.invchi2(fscore,len(features)*2)

    # 对数卡方分布
    def invchi2(self,chi, df):
        m = chi / 2.0
        sum = term = math.exp(-m)
        for i in range(1, df//2):
            term *= m / i
            sum += term
        return min(sum, 1.0)    

    def setminimum(self,cat,min):
        self.minimums[cat] = min

    def getminimum(self,cat):
        if cat not in self.minimums: return 0
        return self.minimums[cat]

    # 概率独立,满足对数卡方分布
    # 如果出现大概率高的值，该分类为最佳分类
    def classify(self,item,default=None):
        # Loop through looking for the best result
        best = default
        max = 0.0
        for c in self.categories():
            p = self.fisherprob(item,c)
            # Make sure it exceeds its minimum
            if p > self.getminimum(c) and p > max:
                best = c
                max = p
        return best

In [ ]:
cl = fisherclassifier(separatewords)
sampletrain(cl,rootdir)

In [ ]:
cl.cprob('台北市'.decode('UTF-8','ignore'),'百科'.decode('UTF-8','ignore'))


In [ ]:
cl.cprob('台北市'.decode('UTF-8','ignore'),'IT'.decode('UTF-8','ignore'))


In [ ]:
cl.cprob('台北市'.decode('UTF-8','ignore'),'人生'.decode('UTF-8','ignore'))


In [ ]:
cl.cprob('熏肉'.decode('UTF-8','ignore'),'人生'.decode('UTF-8','ignore'))


In [ ]:
cl.cprob('又称烟熏食品香精'.decode('UTF-8','ignore'),'食品'.decode('UTF-8','ignore'))



In [ ]:
judge = cl.classify('怎样区分罗氏虾、基围虾、大头虾、青虾、九节虾、花虾、竹节虾、对虾、沙虾、虎虾……各种虾 又称烟熏食品香精 烟熏调味品可分为原生型和派生型两大类 但传统烟熏法温度很难控制 怎样区分大闸蟹、毛蟹、青蟹、梭子蟹、河蟹、红鲟、螃蜞，帝王蟹，松叶蟹等等螃蟹'.decode('UTF-8','ignore'),default='unknown')
print judge



In [ ]:
judge = cl.classify('''
支付意愿法。简单说，就是研究者进行问卷调查统计，如果能够减少你百分之一的死亡率，你愿意付多少钱？这个方法算出来的数字普遍比第一种要高，不过人们在回答问卷时对假想风险的评定往往与面临真实死亡风险时的反映有所不同，所以可能不如第一种客观。

第三种方法是人力资本价值法。也就是开头提到的按工资来计算人命价值。这个方法的最大缺陷是低估了人的生命价值，因为生命的意义不仅仅是工作和赚工资。比如一个乞丐一生没有收入，他的生命是不是就没有价值呢？显然不是。

为生命定价的主要目的是用于政策研究。比如，美国官方各个部门对生命的定价都不一样，美国环保部的定价是910万，食品药品监督局是790万，交通部是940万。以这个定价为标准，很多政策就很容易决定。比如修缮高速公路可以减少10个交通事故死亡人数，需要花1亿美元，交通部一算，十个人值9400万美元，就不会去修这条路。

问题中的法国巴黎人和叙利亚人，生命价格必然是不一样的。考虑到美国和中国的人命价格差距已经相差约50倍，那么法国首都和叙利亚前线的人命价格恐怕相差会更多。

你看，人的生命在经济学家眼里比法律工作者那里更贵。而比起政客来，大家都显得情深意重。举个例子，大家都知道烟草对身体有害，吸烟致男性平均减寿7.13年，女性平均减寿4.5年。有人说，不禁烟是因为国家需要收税，可是事实上，政府在烟草上的税收，是少于因为吸烟造成的医疗开销和生产率损失的。那么，既然对健康不利同时又消耗医疗资源，为什么政府不干脆禁烟呢？


''',default='unknown')
print judge

In [ ]:
judge = cl.classify('熏肉'.decode('UTF-8','ignore'),default='unknown')
print judge


# 共用数据库

In [ ]:
cl = fisherclassifier(separatewords)
cl.setdb('zhihu-fisher.db')

In [ ]:
sampletrain(cl,rootdir)

In [ ]:
sampletrain(cl)
cl.cprob('quick','good')
cl.cprob('money','good')
cl.cprob('money','bad')
cl.weightedprob('money','bad',cl.cprob)
cl.fisherprob('money','good')
cl.fisherprob('money','bad')

cl.classify('quick rabbit')
cl.classify('quick money')

cl.setminimum('bad',0.8)
cl.classify('quick money')

cl.setminimum('good',0.4)
cl.classify('quick rabbit')

cl = fisherclassifier(getwords)
cl.setdb('fisher.db')
sampletrain(cl)

c2 = naivebayes(getwords)
c2.setdb('fisher.db')
c2.classify('quick money')
cl.classify('quick money')

In [ ]:
cl = naivebayes(separatewords)
cl.setdb('zhihu-fisher.db')